# PinSAGE-style Sampling for Ranking (Goodbooks-10k, Graph3)

In this notebook we implement a PinSAGE-style recommendation training loop:

random-walk based neighbor sampling on the bipartite user–item graph,

mini-batch training with subgraphs built from sampled neighborhoods,

BPR loss aligned with ranking metrics.

## Why PinSAGE-style?

Previous models used uniform k-hop neighbor sampling (NeighborLoader). PinSAGE introduces a more recommender-centric sampling strategy:

sample neighborhoods using random walks (captures co-visitation / collaborative signals),

optionally apply importance weighting (in full PinSAGE),

train a GNN with a ranking loss.

## Setup

Graph: user–book bipartite (train interactions only).

Sampling: random walks from seed users/items to generate a “relevant neighborhood pool”.

Model: GraphSAGE (PinSAGE typically uses SAGE-like aggregators) trained with BPR.

Evaluation: LOO candidate-based ranking (C=1000 / C=2000).

In [1]:
# ============================
# Cell 1: Imports + device + paths
# ============================

import os
import json
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

from collections import defaultdict
from dataclasses import dataclass
from pathlib import Path
from tqdm.auto import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

PROJECT_ROOT = Path(r"D:/ML/GNN/graph_recsys")
ARTIFACTS = PROJECT_ROOT / "artifacts" / "v2_proper"
BUNDLE_DIR = ARTIFACTS / "graph3_bundle"

print("BUNDLE_DIR:", BUNDLE_DIR)
assert BUNDLE_DIR.exists(), f"Missing bundle: {BUNDLE_DIR}"

DEVICE: cuda
BUNDLE_DIR: D:\ML\GNN\graph_recsys\artifacts\v2_proper\graph3_bundle


In [2]:
# ============================
# Cell 2: Load LOO splits
# ============================

z = np.load(BUNDLE_DIR / "splits_ui.npz", allow_pickle=True)

train_ui = z["train_ui"].astype(np.int64)
val_ui   = z["val_ui"].astype(np.int64)
test_ui  = z["test_ui"].astype(np.int64)

U = int(z["U"])
B = int(z["B"])

print("train/val/test:", train_ui.shape, val_ui.shape, test_ui.shape)
print("U, B:", U, B)

train/val/test: (4926384, 2) (53398, 2) (53398, 2)
U, B: 53398 9999


In [3]:
# ============================
# Cell 3: train_pos + gt + leak check
# ============================

train_pos = defaultdict(set)
for u, i in train_ui:
    train_pos[int(u)].add(int(i))

val_gt  = {int(u): int(i) for u, i in val_ui}
test_gt = {int(u): int(i) for u, i in test_ui}

leaks_val = sum(1 for u, i in val_gt.items() if i in train_pos[u])
leaks_test = sum(1 for u, i in test_gt.items() if i in train_pos[u])

print("train_pos users:", len(train_pos))
print("[val] leaks:", leaks_val, "/", len(val_gt))
print("[test] leaks:", leaks_test, "/", len(test_gt))

train_pos users: 53398
[val] leaks: 0 / 53398
[test] leaks: 0 / 53398


In [4]:
# ============================
# Cell 4: Build adjacency lists for fast random walks
# Notes:
# - We store user->items and item->users as numpy arrays for fast sampling.
# ============================

user2items = [None] * U
item2users = [None] * B

tmp_u = [[] for _ in range(U)]
tmp_i = [[] for _ in range(B)]

for u, i in train_ui:
    tmp_u[int(u)].append(int(i))
    tmp_i[int(i)].append(int(u))

for u in range(U):
    user2items[u] = np.asarray(tmp_u[u], dtype=np.int64)
for i in range(B):
    item2users[i] = np.asarray(tmp_i[i], dtype=np.int64)

deg_u = np.array([len(x) for x in user2items], dtype=np.int32)
deg_i = np.array([len(x) for x in item2users], dtype=np.int32)

print("user deg min/mean/max:", deg_u.min(), deg_u.mean(), deg_u.max())
print("item deg min/mean/max:", deg_i.min(), deg_i.mean(), deg_i.max())

user deg min/mean/max: 3 92.25783737218623 197
item deg min/mean/max: 4 492.6876687668767 19452


In [18]:
# ============================
# Cell 5: PinSAGE-style sampler via random walks
# Goal:
# - For each seed user, run multiple short random walks on the bipartite graph
# - Collect visited items as a relevance-biased neighborhood pool
# ============================

CFG = {
    "embedding_dim": 64,
    "num_layers": 2,
    "dropout": 0.1,
    "lr": 1e-3,
    "weight_decay": 1e-6,
    "epochs": 30,
    "batch_size_users": 1024,
    # PinSAGE-style sampling params:
    "num_walks": 10,
    "walk_len": 4,          # even length: user->item->user->item...
    "items_per_user": 80,   # cap visited item pool per seed user
    # BPR:
    "bpr_reg": 1e-6,
    # Eval:
    "seed": 42,
    "patience": 7,
    "min_delta": 1e-4,
}
CFG

{'embedding_dim': 64,
 'num_layers': 2,
 'dropout': 0.1,
 'lr': 0.001,
 'weight_decay': 1e-06,
 'epochs': 30,
 'batch_size_users': 1024,
 'num_walks': 10,
 'walk_len': 4,
 'items_per_user': 80,
 'bpr_reg': 1e-06,
 'seed': 42,
 'patience': 7,
 'min_delta': 0.0001}

In [19]:
# ============================
# Cell 5b: Random-walk based item pool for a batch of users
# ============================

SEED = CFG["seed"]
rng = np.random.default_rng(SEED)

def sample_item_pool_for_users(users_np: np.ndarray,
                               num_walks: int,
                               walk_len: int,
                               items_cap: int):
    """
    Returns:
      pools: list of np.array item_ids per user (unique, capped)
    """
    pools = []
    for u in users_np:
        u = int(u)
        visited = []

        for _ in range(num_walks):
            cur_u = u
            for step in range(walk_len):
                # user -> item
                items = user2items[cur_u]
                if items.size == 0:
                    break
                it = int(items[rng.integers(0, items.size)])
                visited.append(it)

                # item -> user
                users = item2users[it]
                if users.size == 0:
                    break
                cur_u = int(users[rng.integers(0, users.size)])

        if len(visited) == 0:
            pools.append(np.empty((0,), dtype=np.int64))
            continue

        uniq = np.unique(np.asarray(visited, dtype=np.int64))
        if uniq.size > items_cap:
            uniq = rng.choice(uniq, size=items_cap, replace=False)
        pools.append(uniq)

    return pools

In [20]:
# ============================
# Cell 6: Build an induced subgraph for a batch
# Nodes in batch-graph:
# - seed users
# - pooled items from random walks
# Edges:
# - all train edges between these users and these items (undirected)
# ============================

def build_batch_subgraph(seed_users: np.ndarray, item_pools):
    """
    Returns:
      n_id_global: torch.LongTensor of global node ids (users 0..U-1, items U..U+B-1)
      edge_index_local: torch.LongTensor [2, E] with local indices
      local_index: dict-like mapping global_id -> local_id (implemented via tensor map)
      seed_users_local: torch.LongTensor local ids of seed users
    """
    # collect global ids
    seed_users = seed_users.astype(np.int64)
    items = np.concatenate(item_pools) if len(item_pools) else np.empty((0,), dtype=np.int64)
    items = np.unique(items)

    # global node ids: users + items(offset by U)
    global_users = seed_users
    global_items = items + U

    n_id = np.concatenate([global_users, global_items]).astype(np.int64)
    n_id_t = torch.from_numpy(n_id).long()

    # build mapping global->local via tensor map (size = num_nodes_ui)
    # (note: num_nodes_ui = U+B)
    idx_map = torch.full((U + B,), -1, dtype=torch.long)
    idx_map[n_id_t] = torch.arange(n_id_t.numel(), dtype=torch.long)

    # build edges: for each seed user, connect to its train items that are in selected item set
    item_set = set(items.tolist())

    rows = []
    cols = []
    for uu in global_users:
        uu = int(uu)
        ui = user2items[uu]
        if ui.size == 0:
            continue
        mask = np.isin(ui, items)  # keep only pooled items
        kept = ui[mask]
        if kept.size == 0:
            continue

        u_loc = int(idx_map[uu])
        for it in kept:
            it_glob = int(it + U)
            it_loc = int(idx_map[it_glob])
            if it_loc >= 0:
                rows.append(u_loc); cols.append(it_loc)
                rows.append(it_loc); cols.append(u_loc)

    if len(rows) == 0:
        edge_index = torch.empty((2, 0), dtype=torch.long)
    else:
        edge_index = torch.tensor([rows, cols], dtype=torch.long)

    seed_users_local = idx_map[torch.from_numpy(global_users)].long()

    return n_id_t, edge_index, idx_map, seed_users_local

In [21]:
# ============================
# Cell 7: Pos/neg sampling (PinSAGE-style tweak)
# Notes:
# - positives: sample from train_pos
# - negatives: sample from items NOT in train_pos
# - optionally bias negatives to be "harder" by sampling from pool first
# ============================

train_pos_arr = {}
for uu in range(U):
    train_pos_arr[uu] = np.fromiter(train_pos[uu], dtype=np.int64)

def sample_positives(users_np: np.ndarray):
    pos = np.empty_like(users_np, dtype=np.int64)
    for idx, uu in enumerate(users_np):
        arr = train_pos_arr[int(uu)]
        pos[idx] = int(arr[rng.integers(0, arr.size)])
    return pos

def sample_negatives(users_np: np.ndarray, item_pools=None, hard_pool_prob=0.5):
    neg = np.empty_like(users_np, dtype=np.int64)
    for idx, uu in enumerate(users_np):
        seen = train_pos[int(uu)]

        # with some prob, try sample from pool (harder negatives)
        if item_pools is not None and item_pools[idx].size > 0 and rng.random() < hard_pool_prob:
            pool = item_pools[idx]
            for _ in range(10):
                j = int(pool[rng.integers(0, pool.size)])
                if j not in seen:
                    neg[idx] = j
                    break
            else:
                # fallback
                while True:
                    j = int(rng.integers(0, B))
                    if j not in seen:
                        neg[idx] = j
                        break
        else:
            while True:
                j = int(rng.integers(0, B))
                if j not in seen:
                    neg[idx] = j
                    break

    return neg

def bpr_loss(u_emb, p_emb, n_emb, reg=0.0):
    pos_scores = (u_emb * p_emb).sum(dim=-1)
    neg_scores = (u_emb * n_emb).sum(dim=-1)
    loss = -torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-8).mean()
    if reg > 0:
        loss = loss + reg * (u_emb.pow(2).mean() + p_emb.pow(2).mean() + n_emb.pow(2).mean())
    return loss

In [22]:
# ============================
# Cell 8: PinSAGE-style model (GraphSAGE)
# Notes:
# - PinSAGE typically uses SAGE-like aggregator; key is sampling strategy.
# ============================

class PinSAGERec(nn.Module):
    def __init__(self, num_nodes: int, dim: int, num_layers: int, dropout: float):
        super().__init__()
        self.emb = nn.Embedding(num_nodes, dim)
        nn.init.normal_(self.emb.weight, std=0.1)

        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(SAGEConv(dim, dim, aggr="mean"))

        self.dropout = dropout
        self.dim = dim

    def forward(self, n_id, edge_index):
        h = self.emb(n_id.to(DEVICE))
        edge_index = edge_index.to(DEVICE)

        for conv in self.convs:
            h = conv(h, edge_index)
            h = F.relu(h)
            h = F.dropout(h, p=self.dropout, training=self.training)
        return h

model = PinSAGERec(num_nodes=U+B, dim=CFG["embedding_dim"], num_layers=CFG["num_layers"], dropout=CFG["dropout"]).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])

print(model)

PinSAGERec(
  (emb): Embedding(63397, 64)
  (convs): ModuleList(
    (0-1): 2 x SAGEConv(64, 64, aggr=mean)
  )
)


In [26]:
# ============================
# Cell 9 (fully fixed): Train one epoch (PinSAGE-style) + skipped tracking + device-safe indexing
# Returns:
# - avg_loss: average BPR loss over processed batches
# - steps: number of processed batches (with edges)
# - skipped: number of skipped batches (no edges)
# ============================

train_users_all = np.arange(U, dtype=np.int64)

def train_one_epoch():
    model.train()

    rng.shuffle(train_users_all)
    bs = CFG["batch_size_users"]

    total_loss = 0.0
    steps = 0
    skipped = 0

    for start in tqdm(range(0, U, bs), desc="train"):
        users_np = train_users_all[start:start + bs]
        if users_np.size == 0:
            continue

        # 1) sample random-walk item pools
        pools = sample_item_pool_for_users(
            users_np,
            num_walks=CFG["num_walks"],
            walk_len=CFG["walk_len"],
            items_cap=CFG["items_per_user"]
        )

        # 2) build induced subgraph (users + pooled items)
        n_id, edge_index, idx_map, seed_users_local = build_batch_subgraph(users_np, pools)

        # If no edges, skip
        if edge_index.numel() == 0:
            skipped += 1
            continue

        # 3) sample positives / negatives (hard negatives from pool with prob=0.7)
        pos_items = sample_positives(users_np)
        neg_items = sample_negatives(users_np, item_pools=pools, hard_pool_prob=0.7)

        # global node ids for items (shift by U)
        pos_nodes = torch.from_numpy(pos_items).long() + U
        neg_nodes = torch.from_numpy(neg_items).long() + U

        # 4) forward on batch subgraph
        # model moves n_id/edge_index to DEVICE internally
        h = model(n_id, edge_index)

        # 5) local indices:
        # idx_map is created on CPU (in build_batch_subgraph), so index it with CPU tensors, then move to DEVICE
        u_loc = seed_users_local.to(DEVICE)

        p_loc = idx_map[pos_nodes.cpu()].to(DEVICE)
        n_loc = idx_map[neg_nodes.cpu()].to(DEVICE)

        u_emb = h[u_loc]

        # 6) helper: fetch item embeddings from subgraph or fallback to raw embedding table
        def get_item_emb(loc_idx, global_nodes):
            """
            loc_idx: LongTensor on DEVICE (local indices in h, -1 means missing from subgraph)
            global_nodes: LongTensor (global node ids), can be CPU
            Returns: [N, dim] tensor on DEVICE
            """
            global_nodes = global_nodes.to(DEVICE)   # move FIRST to avoid device-mismatch
            loc_idx = loc_idx.to(DEVICE)

            mask = loc_idx >= 0
            out = torch.empty((global_nodes.size(0), CFG["embedding_dim"]), device=DEVICE)

            if mask.any():
                out[mask] = h[loc_idx[mask]]
            if (~mask).any():
                out[~mask] = model.emb(global_nodes[~mask])

            return out

        p_emb = get_item_emb(p_loc, pos_nodes)
        n_emb = get_item_emb(n_loc, neg_nodes)

        # 7) BPR loss + step
        loss = bpr_loss(u_emb, p_emb, n_emb, reg=CFG["bpr_reg"])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += float(loss.detach().cpu())
        steps += 1

    avg_loss = total_loss / max(1, steps)
    return avg_loss, steps, skipped

In [27]:
# ============================
# Cell 10: Candidate-based LOO evaluation (fast)
# ============================

@torch.no_grad()
def eval_loo_sampled(model, gt_dict, users_subset, C=1000, Ks=(10, 20, 50), seed=42):
    model.eval()
    rng_local = np.random.default_rng(seed)

    hits = {k: 0 for k in Ks}
    ndcgs = {k: 0.0 for k in Ks}

    emb = model.emb.weight.detach()

    for u in tqdm(users_subset, desc=f"eval(C={C})"):
        gt = int(gt_dict[int(u)])
        seen = train_pos[int(u)]

        negs = []
        while len(negs) < C - 1:
            j = int(rng_local.integers(0, B))
            if (j not in seen) and (j != gt):
                negs.append(j)

        cand_items = np.array([gt] + negs, dtype=np.int64)
        cand_nodes = torch.from_numpy(cand_items).long().to(DEVICE) + U

        u_node = torch.tensor([int(u)], device=DEVICE, dtype=torch.long)
        u_vec = emb[u_node]
        i_vec = emb[cand_nodes]
        scores = (u_vec * i_vec).sum(dim=-1)

        rank = torch.argsort(scores, descending=True)
        gt_pos = (rank == 0).nonzero(as_tuple=False).item()

        for k in Ks:
            if gt_pos < k:
                hits[k] += 1
                ndcgs[k] += 1.0 / math.log2(gt_pos + 2)

    n = len(users_subset)
    out = {f"Hit@{k}": hits[k] / n for k in Ks}
    out.update({f"NDCG@{k}": ndcgs[k] / n for k in Ks})
    return out

subset_2k = np.random.default_rng(SEED).choice(np.arange(U), size=2000, replace=False)
subset_10k = np.random.default_rng(SEED).choice(np.arange(U), size=10000, replace=False)

print("Smoke eval (C=200):", eval_loo_sampled(model, val_gt, subset_2k, C=200, Ks=(10,20,50), seed=SEED))

eval(C=200):   0%|          | 0/2000 [00:00<?, ?it/s]

Smoke eval (C=200): {'Hit@10': 0.046, 'Hit@20': 0.096, 'Hit@50': 0.239, 'NDCG@10': 0.020917185507600185, 'NDCG@20': 0.033487712151199264, 'NDCG@50': 0.061197168968758715}


In [29]:
# ============================
# Cell 11 (fixed): Train loop + early stopping + steps/skipped logging
# Metric for early stopping: val NDCG@10 @ (C=1000, 10k users)
# ============================

@dataclass
class EarlyStopper:
    patience: int = 7
    min_delta: float = 1e-4
    best: float = -1e9
    best_epoch: int = -1
    bad_count: int = 0
    best_state: dict = None

    def step(self, metric_value: float, model: torch.nn.Module, epoch: int) -> bool:
        improved = metric_value > (self.best + self.min_delta)
        if improved:
            self.best = metric_value
            self.best_epoch = epoch
            self.bad_count = 0
            self.best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        else:
            self.bad_count += 1
        return self.bad_count >= self.patience

    def load_best(self, model: torch.nn.Module, device=DEVICE):
        model.load_state_dict({k: v.to(device) for k, v in self.best_state.items()})


EARLY = EarlyStopper(patience=CFG["patience"], min_delta=CFG["min_delta"])

history = []
for ep in range(1, CFG["epochs"] + 1):
    # train_one_epoch now returns (avg_loss, steps, skipped)
    avg_loss, steps, skipped = train_one_epoch()

    m200 = eval_loo_sampled(model, val_gt, subset_2k, C=200, Ks=(10,20,50), seed=SEED)
    m1000 = eval_loo_sampled(model, val_gt, subset_10k, C=1000, Ks=(10,20,50), seed=SEED)
    val_ndcg10 = float(m1000["NDCG@10"])

    history.append({
        "epoch": ep,
        "loss": avg_loss,
        "steps": steps,
        "skipped": skipped,
        **{f"val200_{k}": float(v) for k, v in m200.items()},
        **{f"val1000_{k}": float(v) for k, v in m1000.items()},
    })

    print(
        f"epoch={ep:02d} loss={avg_loss:.4f} | steps={steps} skipped={skipped} | "
        f"val(C=200) NDCG@10={m200['NDCG@10']:.5f} | "
        f"val(C=1000) NDCG@10={val_ndcg10:.6f}"
    )

    if EARLY.step(val_ndcg10, model, ep):
        print(f"Early stopping at epoch {ep}. Best epoch={EARLY.best_epoch} best NDCG@10={EARLY.best:.6f}")
        break

EARLY.load_best(model, device=DEVICE)
print("Loaded best checkpoint:", EARLY.best_epoch, "best val NDCG@10:", EARLY.best)

train:   0%|          | 0/53 [00:00<?, ?it/s]

eval(C=200):   0%|          | 0/2000 [00:00<?, ?it/s]

eval(C=1000):   0%|          | 0/10000 [00:00<?, ?it/s]

epoch=01 loss=0.6826 | steps=53 skipped=0 | val(C=200) NDCG@10=0.02897 | val(C=1000) NDCG@10=0.006591


train:   0%|          | 0/53 [00:00<?, ?it/s]

...

epoch=29 loss=0.5001 | steps=53 skipped=0 | val(C=200) NDCG@10=0.11476 | val(C=1000) NDCG@10=0.037505


train:   0%|          | 0/53 [00:00<?, ?it/s]

eval(C=200):   0%|          | 0/2000 [00:00<?, ?it/s]

eval(C=1000):   0%|          | 0/10000 [00:00<?, ?it/s]

epoch=30 loss=0.4980 | steps=53 skipped=0 | val(C=200) NDCG@10=0.11508 | val(C=1000) NDCG@10=0.038234
Loaded best checkpoint: 30 best val NDCG@10: 0.03823351739798701


In [30]:
# ============================
# Cell 12: Final TEST eval + save artifacts
# ============================

test_subset_10k = np.random.default_rng(SEED + 123).choice(np.arange(U), size=10000, replace=False)

test_m1000 = eval_loo_sampled(model, test_gt, test_subset_10k, C=1000, Ks=(10,20,50), seed=SEED + 123)
print("TEST (C=1000, 10k users):", test_m1000)

test_m2000 = eval_loo_sampled(model, test_gt, test_subset_10k, C=2000, Ks=(10,20,50), seed=SEED + 123)
print("TEST (C=2000, 10k users):", test_m2000)

hist_df = pd.DataFrame(history)
out_dir = ARTIFACTS / "ablation_runs" / "pinsage_style_sampling"
out_dir.mkdir(parents=True, exist_ok=True)

hist_path = out_dir / "history_pinsage_style_bpr.csv"
hist_df.to_csv(hist_path, index=False)

meta = {
    "best_epoch": EARLY.best_epoch,
    "best_val_ndcg10_C1000_10k": float(EARLY.best),
    "config": CFG,
    "bundle_dir": str(BUNDLE_DIR),
    "test_C1000": test_m1000,
    "test_C2000": test_m2000,
}
with open(out_dir / "run_meta.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

ckpt_path = out_dir / "pinsage_style_best.pt"
torch.save({"state_dict": EARLY.best_state, "meta": meta}, ckpt_path)

print("Saved:", hist_path)
print("Saved:", ckpt_path)
print("Saved:", out_dir / "run_meta.json")

eval(C=1000):   0%|          | 0/10000 [00:00<?, ?it/s]

TEST (C=1000, 10k users): {'Hit@10': 0.0761, 'Hit@20': 0.1164, 'Hit@50': 0.2019, 'NDCG@10': 0.041187646994723576, 'NDCG@20': 0.051263158758467424, 'NDCG@50': 0.06817057082797173}


eval(C=2000):   0%|          | 0/10000 [00:00<?, ?it/s]

TEST (C=2000, 10k users): {'Hit@10': 0.0493, 'Hit@20': 0.0754, 'Hit@50': 0.1341, 'NDCG@10': 0.026857281486186428, 'NDCG@20': 0.033402968913480185, 'NDCG@50': 0.04495259872554541}
Saved: D:\ML\GNN\graph_recsys\artifacts\v2_proper\ablation_runs\pinsage_style_sampling\history_pinsage_style_bpr.csv
Saved: D:\ML\GNN\graph_recsys\artifacts\v2_proper\ablation_runs\pinsage_style_sampling\pinsage_style_best.pt
Saved: D:\ML\GNN\graph_recsys\artifacts\v2_proper\ablation_runs\pinsage_style_sampling\run_meta.json


# Results & Conclusions (PinSAGE-style Sampling)

## What we built
In this notebook we implemented a **PinSAGE-style** training loop for recommendation on the Goodbooks-10k **Graph3** setup:
- We use the **train-only user–item bipartite graph**.
- For each mini-batch of seed users we generate **neighborhoods via random walks** (PinSAGE idea).
- We build an **induced subgraph** from the visited items and train a **GraphSAGE** model on that subgraph.
- Objective: **BPR loss** (ranking-aligned, consistent with Hit@K / NDCG@K evaluation).

This differs from standard neighbor sampling because the neighborhood is **biased by random walks**, which tends to capture collaborative/co-visitation structure.

## Training sanity checks
- Induced subgraphs were valid: `skipped=0` across all epochs (no empty subgraphs).
- Loss decreased steadily and validation ranking improved consistently.

## Best checkpoint
- Best epoch: **30**
- Best validation (C=1000, 10k users): **NDCG@10 = 0.03823**

## Final test metrics (10k users)
**Candidate evaluation C=1000**
- Hit@10 = **0.0761**
- Hit@20 = **0.1164**
- Hit@50 = **0.2019**
- NDCG@10 = **0.04119**
- NDCG@20 = **0.05126**
- NDCG@50 = **0.06817**

**Candidate evaluation C=2000**
- Hit@10 = **0.0493**
- Hit@20 = **0.0754**
- Hit@50 = **0.1341**
- NDCG@10 = **0.02686**
- NDCG@20 = **0.03340**
- NDCG@50 = **0.04495**

## Key takeaway
PinSAGE-style random-walk neighborhoods provide a strong training signal for ranking and produce **competitive results** compared to other sampling-based GNN recommenders.

## Saved artifacts
All artifacts are saved under:
`artifacts/v2_proper/ablation_runs/pinsage_style_sampling/`
- `history_pinsage_style_bpr.csv`
- `pinsage_style_best.pt`
- `run_meta.json`

## Next step
Proceed to **09_final_eval_and_report.ipynb**:
- load all `run_meta.json` from `ablation_runs/`,
- build a single comparison table for all models,
- highlight the best model(s) and produce the final report.